In [2]:
#!/usr/bin/env python
import snowflake.connector

import time
import calendar

import hashlib as hashlib
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import sklearn
import sklearn.metrics as sm

from sklearn import datasets
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.preprocessing import OneHotEncoder
from sklearn.cluster import KMeans
from sklearn.preprocessing import scale
from mpl_toolkits.mplot3d import Axes3D

from snowflake.connector.pandas_tools import write_pandas
from snowflake.connector.pandas_tools import pd_writer
from hashlib import md5

from datetime import date, datetime, timedelta
from dateutil.parser import parse as dateParse

In [ ]:
#---------------------------------------
# For installing dependencies
#---------------------------------------
import sys
!{sys.executable} -m pip install "snowflake-connector-python[pandas]"
!{sys.executable} -m pip install "matplotlib"
!{sys.executable} -m pip install "scikit-learn"

In [3]:
#---------------------------------------
# CONNECT TO SNOWFLAKE
#---------------------------------------

con = snowflake.connector.connect(
    user='codestudio_samplefarm_evening',
    password='ikRsb9D00UNr',
    account='FT76388.ap-southeast-2',
    #warehouse='Main',
    database='AW_RESEARCH_STAGING',
    schema='SAMPLE_FARM_EVENING'
    )

In [ ]:
#---------------------------------------
# Test reaching the dataset
#---------------------------------------
con_cursor = con.cursor()
try:
    con_cursor.execute("SELECT * FROM AW_RESEARCH_STAGING.SAMPLE_FARM_EVENING.FARM_DETAILS")
    
    pd.set_option("max_rows", 100)
    
    df = con_cursor.fetch_pandas_all()
    
    print(df)
    
finally:
    con_cursor.close()

In [33]:
#---------------------------------------
# Test clustering algorithm
#---------------------------------------
con_cursor = con.cursor()

try:
    # List of regions to make personas for
    regions = ['Summer High Rainfall NSW / QLD', 
                'Summer Medium Rainfall NSW / QLD',
                'Low Rainfall NSW / QLD',
                'High Rainfall NSW',
                'Medium Rainfall NSW',
                'High Rainfall VIC / SA',
                'Medium Rainfall VIC / SA',
                'Low Rainfall VIC / SA',
                'High Rainfall TAS',
                'Summer Rainfall TAS',
                'Low Rainfall TAS',
                'Northern Australia Intensive Grazing Systems',
                'Northern Australia Pastoral Grazing',
                'Southern Australia High Rainfall',
                'Southern Australia Low Rainfall',
                'High Rainfall WA',
                'Low Rainfall WA']
    # Dictionary of dataframes that include a prediction column for each region
    clustered_regions = {}

    # Perform clustering on each region, add results to dictionary
    for region in regions:

        con_cursor.execute("SELECT \
        CREATION_DATES,\
        FARM_TOWN,\
        FARM_REGION_STATE,\
        PASTURE_COUNT,\
        TOTAL_ARABLE_LAND_SIZE,\
        AVERAGE_CROP_TYPE,\
        LANDMARK_COUNT,\
        TOP_LANDMARK,\
        INDIVIDUAL_ANIMAL_COUNT,\
        TOP_INDIVIDUAL_ANIMAL_MANAGEMENT_GROUP,\
        TOP_ANIMAL_WEANED_STATUS,\
        TOP_ANIMAL_SEX,\
        WEIGH_RECORD_COUNT_INDIVIDUAL,\
        WEIGH_RECORD_COUNT_GROUP,\
        INDIVIDUAL_WEIGHTS_JAN,\
        INDIVIDUAL_WEIGHTS_FEB,\
        INDIVIDUAL_WEIGHTS_MAR,\
        INDIVIDUAL_WEIGHTS_APR,\
        INDIVIDUAL_WEIGHTS_MAY,\
        INDIVIDUAL_WEIGHTS_JUN,\
        INDIVIDUAL_WEIGHTS_JUL,\
        INDIVIDUAL_WEIGHTS_AUG,\
        INDIVIDUAL_WEIGHTS_SEP,\
        INDIVIDUAL_WEIGHTS_OCT,\
        INDIVIDUAL_WEIGHTS_NOV,\
        INDIVIDUAL_WEIGHTS_DEC,\
        GROUP_WEIGHTS_JAN,\
        GROUP_WEIGHTS_FEB,\
        GROUP_WEIGHTS_MAR,\
        GROUP_WEIGHTS_APR,\
        GROUP_WEIGHTS_MAY,\
        GROUP_WEIGHTS_JUN,\
        GROUP_WEIGHTS_JUL,\
        GROUP_WEIGHTS_AUG,\
        GROUP_WEIGHTS_SEP,\
        GROUP_WEIGHTS_OCT,\
        GROUP_WEIGHTS_NOV,\
        GROUP_WEIGHTS_DEC\
        \
        FROM AW_RESEARCH_STAGING.SAMPLE_FARM_EVENING.FARM_DETAILS\
        \
        WHERE FARM_REGION = (%s)", (region))

        df = con_cursor.fetch_pandas_all()

        # Break if no farms found for region
        if df.empty:
            break 

        #---------------
        # Preprocessing
        #---------------
        # Encoding categorical variables
        df = pd.get_dummies(df, columns=
            ["FARM_TOWN", 
            "AVERAGE_CROP_TYPE", 
            "TOP_LANDMARK", 
            "TOP_INDIVIDUAL_ANIMAL_MANAGEMENT_GROUP", 
            "TOP_ANIMAL_WEANED_STATUS", 
            "TOP_ANIMAL_SEX"], 
            drop_first=True)
        
        # Replace NaN values with 0 by default
        df.fillna(0, inplace=True)
        
        #print(df)

        clustering_results = KMeans(n_clusters=2, random_state=10).fit_predict(df)

        # Append predictions
        df['PERSONA_PREDICTIONS'] = pd.Series(clustering_results, index=df.index)

        clustered_regions[region] = df

    print(clustered_regions)
finally:
    con_cursor.close()

{'Summer High Rainfall NSW / QLD':      CREATION_DATES  FARM_REGION_STATE  PASTURE_COUNT  TOTAL_ARABLE_LAND_SIZE  \
0     1499809500827                  0            390              22435.1400   
1     1551935603789                  0              0                  0.0000   
2     1558590184634                  0              0                  0.0000   
3     1496714638425                  0             99               3487.0800   
4     1506307596199                  0              1                 32.7300   
..              ...                ...            ...                     ...   
481   1480480108810                  0           4758             311577.2400   
482   1527557934580                  0             16                515.9153   
483   1501822703590                  0              0                  0.0000   
484   1493185465248                  0            777              42050.8200   
485   1529825462667                  0              2                389.1